In [ ]:
import pandas as pd
import numpy as np

# Load the stock prices CSV file (adjust the path as needed)
file_path = "stock_prices_5m.csv"
df = pd.read_csv(file_path) 

# If a Date column exists, parse it as datetime and sort by time
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
    df.reset_index(drop=True, inplace=True) 

# Ensure sorting by Ticker symbol first, then by Date
df.sort_values(by=['Ticker', 'Datetime'], inplace=True)
df.reset_index(drop=True, inplace=True)

#############################################
# 1. MACD Calculation
#############################################
df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA12'] - df['EMA26']
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['Hist'] = df['MACD'] - df['Signal']

#############################################
# 2. RSI Calculation (Period = 14)
#############################################
delta = df['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
window_length = 14
avg_gain = gain.rolling(window=window_length, min_periods=window_length).mean()
avg_loss = loss.rolling(window=window_length, min_periods=window_length).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

#############################################
# 3. Stochastic Oscillator (Stochs)
#############################################
stoch_period = 14
df['Lowest_Low'] = df['Low'].rolling(window=stoch_period).min()
df['Highest_High'] = df['High'].rolling(window=stoch_period).max()
df['%K'] = 100 * (df['Close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])
df['%D'] = df['%K'].rolling(window=3).mean()

#############################################
# 4. ADX Calculation (Average Directional Index)
#############################################
adx_period = 14
df['TR'] = np.maximum(df['High'] - df['Low'], np.maximum(abs(df['High'] - df['Close'].shift(1)), abs(df['Low'] - df['Close'].shift(1))))
df['+DM'] = np.where((df['High'] - df['High'].shift(1)) > (df['Low'].shift(1) - df['Low']), df['High'] - df['High'].shift(1), 0)
df['-DM'] = np.where((df['Low'].shift(1) - df['Low']) > (df['High'] - df['High'].shift(1)), df['Low'].shift(1) - df['Low'], 0)

df['TR14'] = df['TR'].rolling(window=adx_period).sum()
df['+DM14'] = df['+DM'].rolling(window=adx_period).sum()
df['-DM14'] = df['-DM'].rolling(window=adx_period).sum()

df['+DI14'] = 100 * (df['+DM14'] / df['TR14'])
df['-DI14'] = 100 * (df['-DM14'] / df['TR14'])
df['DX'] = 100 * abs(df['+DI14'] - df['-DI14']) / (df['+DI14'] + df['-DI14'])
df['ADX'] = df['DX'].rolling(window=adx_period).mean()

#############################################
# 5. AROON Calculation
#############################################
aroon_period = 25
df['Aroon_Up'] = df['High'].rolling(window=aroon_period).apply(lambda x: ((aroon_period - x[::-1].idxmax()) / aroon_period) * 100)
df['Aroon_Down'] = df['Low'].rolling(window=aroon_period).apply(lambda x: ((aroon_period - x[::-1].idxmin()) / aroon_period) * 100)

#############################################
# 6. Bollinger Bands Calculation (BOLL)
#############################################
boll_period = 20
df['BOLL_Mid'] = df['Close'].rolling(window=boll_period).mean()
df['BOLL_STD'] = df['Close'].rolling(window=boll_period).std()
df['BOLL_Upper'] = df['BOLL_Mid'] + 2 * df['BOLL_STD']
df['BOLL_Lower'] = df['BOLL_Mid'] - 2 * df['BOLL_STD']

#############################################
# 7. Price Oscillator (OSC)
#############################################
sma_fast = df['Close'].rolling(window=5).mean()
sma_slow = df['Close'].rolling(window=10).mean()
df['OSC'] = (sma_fast - sma_slow) / sma_slow * 100

#############################################
# 8. Buy/Sell Signal Calculation
#############################################
# Generate Buy/Sell signals for all indicators
df['MACD_Buy'] = df['MACD'] > df['Signal']
df['MACD_Sell'] = df['MACD'] < df['Signal']
df['RSI_Buy'] = df['RSI'] < 30
df['RSI_Sell'] = df['RSI'] > 70
df['Stochs_Buy'] = df['%K'] > df['%D']
df['Stochs_Sell'] = df['%K'] < df['%D']
df['ADX_Buy'] = df['ADX'] > 25
df['ADX_Sell'] = df['ADX'] < 25
df['Aroon_Buy'] = df['Aroon_Up'] > df['Aroon_Down']
df['Aroon_Sell'] = df['Aroon_Up'] < df['Aroon_Down']
df['BOLL_Buy'] = df['Close'] <= df['BOLL_Lower']
df['BOLL_Sell'] = df['Close'] >= df['BOLL_Upper']
df['OSC_Buy'] = df['OSC'] > 0
df['OSC_Sell'] = df['OSC'] < 0

# Save results
df.to_csv("stock_prices_with_signals.csv", index=False) 

df


,Datetime,Open,High,Low,Close,Volume,Ticker,EMA12,EMA26,MACD,...,Stochs_Buy,Stochs_Sell,ADX_Buy,ADX_Sell,Aroon_Buy,Aroon_Sell,BOLL_Buy,BOLL_Sell,OSC_Buy,OSC_Sell
0,2025-01-14 14:30:00+00:00,234.750000,235.940002,234.259995,235.889999,2136084.0,AAPL,235.889999,235.889999,0.000000,...,False,False,False,False,False,False,False,False,False,False
1,2025-01-14 14:35:00+00:00,235.876999,236.029999,234.869995,235.020004,740327.0,AAPL,235.756154,235.825555,-0.069401,...,False,False,False,False,False,False,False,False,False,False
2,2025-01-14 14:40:00+00:00,235.016998,235.569901,234.850006,235.544998,460742.0,AAPL,235.723668,235.804773,-0.081105,...,False,False,False,False,False,False,False,False,False,False
3,2025-01-14 14:45:00+00:00,235.544998,235.649994,235.009995,235.623993,574276.0,AAPL,235.708334,235.791382,-0.083048,...,False,False,False,False,False,False,False,False,False,False
4,2025-01-14 14:50:00+00:00,235.630005,235.699997,234.960007,235.179993,371774.0,AAPL,235.627051,235.746094,-0.119044,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22379,2025-03-13 19:35:00+00:00,240.154999,240.500000,239.259995,239.339996,715115.0,TSLA,240.382760,239.837165,0.545595,...,False,True,True,False,False,True,False,False,False,True
22380,2025-03-13 19:40:00+00:00,239.345001,239.940002,238.639999,239.080002,956868.0,TSLA,240.182335,239.781079,0.401257,...,False,True,True,False,False,True,True,False,False,True
22381,2025-03-13 19:45:00+00:00,239.130005,239.639999,238.529999,239.500000,1003163.0,TSLA,240.077361,239.760258,0.317103,...,True,False,True,False,False,True,False,False,False,True
22382,2025-03-13 19:50:00+00:00,239.410004,239.897095,238.151001,239.240005,1348884.0,TSLA,239.948537,239.721721,0.226816,...,True,False,True,False,False,True,False,False,False,True
